In [146]:
import numpy as np

from src import PricingMethod, UnderlyingModel, Derivative

In [214]:
n_time = 10
n_mc = 10000

sigma1 = 0.5
sigma2 = 0.4
sigma3 = 0.8

dic_param_bs = {
    'r' : 0.01,
    'q' : [0.02, 0.1, 0.0],
    'sigma' : np.array([sigma1, sigma2, sigma3]),
    'underlying_price' : [100,80, 210],
    'rho' : np.array([[1.0, 0.6, 0.5], [0.6, 1.0, 0.8], [0.5, 0.8, 1.0]]),
    'n_underlying' : 3 
}

dic_param_call = {
    'K' : [100,90, 210],
    'valuation_time' : 0,
    'end_time' : 1
}

european_call = Derivative.EuropeanCall(dic_param_call)
bs_model = UnderlyingModel.BlackScholes(dic_param_bs)
pricer_mc = PricingMethod.PricingMonteCarlo(n_time,n_mc)

In [215]:
dic_param_bs['sigma'].shape

(3,)

In [216]:
print(bs_model)

Black Scholes model
r: 0.01
q: [0.02, 0.1, 0.0]
sigma: [0.5 0.4 0.8]
underlying_price: [100, 80, 210]
rho: [[1.  0.6 0.5]
 [0.6 1.  0.8]
 [0.5 0.8 1. ]]
n_underlying: 3



In [217]:
correlated_paths = pricer_mc.generate_paths(european_call, bs_model)

In [218]:
np.log(correlated_paths)

array([[4.60517019, 4.47283402, 4.5486276 , ..., 4.45327975, 4.50238798,
        4.19291187],
       [4.38202663, 4.27525714, 4.14229646, ..., 4.2210237 , 4.33221873,
        3.94165686],
       [5.34710753, 5.43621996, 5.23766943, ..., 5.01838595, 5.25117732,
        4.66953718],
       ...,
       [4.60517019, 4.68296045, 4.91705125, ..., 4.3558954 , 4.16858134,
        4.29854947],
       [4.38202663, 4.38920082, 4.55707473, ..., 4.18832914, 4.12778183,
        4.14079831],
       [5.34710753, 5.18646552, 5.45196012, ..., 4.59747965, 4.47092116,
        4.70263016]])

In [219]:
n_mc

10000

In [220]:
dic_idx_assets = {}
for k in range(dic_param_bs['n_underlying']):
    dic_idx_assets[k] =  np.arange(k,n_mc*dic_param_bs['n_underlying'], dic_param_bs['n_underlying'])

In [221]:
mat = np.array([np.ravel(correlated_paths[dic_idx_assets[k],-1]) for k in range(dic_param_bs['n_underlying'])])

In [222]:
np.cov(np.log(mat))

array([[0.25094453, 0.1199176 , 0.2028136 ],
       [0.1199176 , 0.16102481, 0.25720205],
       [0.2028136 , 0.25720205, 0.6418299 ]])

In [223]:
np.diag(dic_param_bs['sigma']) @ dic_param_bs['rho'] @ np.diag(dic_param_bs['sigma'])

array([[0.25 , 0.12 , 0.2  ],
       [0.12 , 0.16 , 0.256],
       [0.2  , 0.256, 0.64 ]])

In [225]:
print(f"error L_inf : {np.max(np.abs( (np.diag(dic_param_bs['sigma']) @ dic_param_bs['rho'] @ np.diag(dic_param_bs['sigma'])) - np.cov(np.log(mat))))}")

error L_inf : 0.0028136025245963647


In [213]:
np.corrcoef(np.log(mat))

array([[1.        , 0.58697918, 0.46630701],
       [0.58697918, 1.        , 0.79452157],
       [0.46630701, 0.79452157, 1.        ]])